In [1]:
from bs4 import BeautifulSoup
from requests_html import HTML
import pandas
import pathlib
import requests
import time

film_links = list()
pages = [f'https://www.ozmovies.com.au/movies/browse?page={x}' for x in range(127)]
for p in pages:
    time.sleep(4)
    r = requests.get(p)
    r = r.text.split('<div class="movie-image pull-left">')[1:]
    for x in r:
        html = HTML(html=x.split('<div class="inline-image">')[0])
        for y in html.links:
            film_links.append(y)

print(len(film_links))

1061


In [2]:
dataframe = pandas.DataFrame(columns=['title', 'year', 'activity', 'agent'])
for x in film_links:
    time.sleep(4)
    r = requests.get(f'https://www.ozmovies.com.au/{x}')
    title = r.text.split('<h2>')[1].split('</h2>')[0]
    year = [k for k in r.text.split('<li class="active">')][1:]
    year = [k.split('</li>')[0] for k in year]
    year = [BeautifulSoup(b, 'lxml').get_text().replace('\n', '').strip() for b in year]
    year = [k for k in year if k in [str(z) for z in range(1880, 2030)]][0]
    r = r.text.split('<div class="movie-primary-details-people pull-left">')[1].split('<div class="pull-right">')[0]
    for a in r.split(' <div class="row">')[1:]:
        activity = a.split('<div class="pull-left personType">')[1].split('</div>')[0]
        agent = a.split('<div class="pull-left personName">')[1].split('<br />')
        agent = [BeautifulSoup(b, 'lxml').get_text().replace('\n', '').strip() for b in agent]
        for h in agent:
            dataframe.loc[len(dataframe)] = [(title), (year), (activity), (h)]

print(len(dataframe))
dataframe.head()

16323


,title,year,activity,agent
0,13 Gantry Row,1998,Key cast:,Rebecca Gibney
1,13 Gantry Row,1998,Key cast:,John Adam
2,13 Gantry Row,1998,Key cast:,Doris Younane
3,13 Gantry Row,1998,Key cast:,Nicholas Hammond
4,13 Gantry Row,1998,Key cast:,Mark Gerber


In [6]:
# this is where you add a label (film + year) and then a work_uuid and an agent_uuid, you may also need to write activty as property
# ie "Key cast:" becomes ontology:hasActor 
import pathlib

print(len(dataframe.title.unique())) # make a label and see what happens

dataframe.to_csv(pathlib.Path.cwd() / 'ozmovies-temp.csv')

1051
